In [1]:
# Import packages
import requests
from bs4 import BeautifulSoup
import math
import re
import time
import codecs
import pandas as pd
import urllib
import os
from google.colab import drive
drive.mount('/content/drive')

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

Mounted at /content/drive


In [2]:
# Define functions
def convert_list_to_dic(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

In [3]:

# Get the HTML code
url = "https://tribunalsdecisions.service.gov.uk/utiac?&page=1"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
soup.prettify()


'<!DOCTYPE html>\n<html>\n <head>\n  <title>\n   Tribunal decisions\n  </title>\n  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n  <link href="/assets/favicon-5aa00a9f9e6dc48d4e92f306a94e9496.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>\n  <!-- For third-generation iPad with high-resolution Retina display: -->\n  <link href="/assets/apple-touch-icon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>\n  <!-- For iPhone with high-resolution Retina display: -->\n  <link href="/assets/apple-touch-icon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>\n  <!-- For first- and second-generation iPad: -->\n  <link href="/assets/apple-touch-icon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>\n  <!-- For non-Retina iPhone, iPod Touch, and Android 2.1+ devices: -->\n  <link href="/assets/apple-touch-icon-57x57.png" rel="apple-touch-icon-precomposed"/>\n  <script>\n   (function(){if(navigator.userAgent.match(/IEM

In [4]:
# Retrieve relevant variables
total_cases = int(str(re.findall(r'(.*)', str(soup.find_all("b")[1]), re.DOTALL))[5:10])
div_pages = int(str(re.findall(r'-(.*)', str(soup.find_all("b")[0]), re.DOTALL))[6:8])
total_pages = math.ceil(total_cases/div_pages)

In [5]:
total_pages

1280

In [6]:
# Save numbers for the future
with open('/content/drive/My Drive/scriptie/total_old_cases.txt', 'w') as f:
    f.write(str(total_cases))   
with open('/content/drive/My Drive/scriptie/total_old_pages.txt', 'w') as f:
    f.write(str(total_pages)) 

In [7]:
# Retrieve **NEW** documents
id_case_list = []
url_case_pdf_list = []
details_cases = []
downloaded_cases = []
total_old_cases = 1215

    
    
with open('/content/drive/My Drive/scriptie/corpus_archive_asylum4.txt', 'w') as f:
    for i in range(0,64):
        print(i)
        url = 'https://tribunalsdecisions.service.gov.uk/utiac?&page=' + str(i+1)
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')  
        table_cases = soup.find_all("table", class_='decisions-table')
        list_cases = re.findall(r'<a href="(.*)"></a>', str(table_cases))
        for j in range(0, len(list_cases)):
            id_case_list.append(list_cases[j])
            url_case = 'https://tribunalsdecisions.service.gov.uk/' + list_cases[j]
            req_case = requests.get(url_case, headers)
            soup_case = BeautifulSoup(req_case.content, 'html.parser') 
            url_case_pdf_aux = str(soup_case.find_all("a", class_='pdf-file'))
            url_case_pdf = re.findall(r'href="(.*)">Download', url_case_pdf_aux)
            url_case_pdf_list.append(url_case_pdf)

            # Get the metadata
            meta_data_case = [item.get_text() for item in soup_case.select("span")]
            # Clean text:
            meta_data_case = [e.replace('\xa0', '') for e in meta_data_case]
            meta_data_case = [e.replace('\n', '') for e in meta_data_case]
            # Append metadata into list of cases
            meta_data_case = convert_list_to_dic(meta_data_case)
            details_cases.append(meta_data_case)

            # Save content on a .txt
            try: 
                text_case_aux = soup_case.find("div", class_='decision-inner').get_text(strip=True)
                f.write('CASE ID: ' + list_cases[j])
                f.write(text_case_aux)
                f.write('\n')
                f.write('\n')
                f.write('\n')
                downloaded_cases.append('OK')
            except:
                url_case_doc_aux = str(soup_case.find_all("a", class_='doc-file'))
                url_case_doc = re.findall(r'href="(.*)">Download', url_case_doc_aux)
                urllib.request.urlretrieve(url_case_doc[0], 'asylum_case_doc')
                
                print("Error!")
                #print(url_case)
                up_boolean = False
                crown_boolean = True
                with codecs.open('asylum_case_doc', 'r', encoding="ISO-8859-1") as f2:
                     for num, line in enumerate(f2, 1):
                        if('Upper Tribunal' in line):
                            up_boolean = True
                        if('CROWN COPYRIGHT' in line):
                            crown_boolean = False
                            f.write(line)
                        if up_boolean and crown_boolean:
                            f.write(line)
                downloaded_cases.append('OK')
             # except:
              #    downloaded_cases.append('FAILED')
               #   print('Impossible to retrieve data on case: ' + url_case)
        #time.sleep(10)
        
        
#os.remove('asylum_case_doc')       
flat_url_case_pdf_list = [item for sublist in url_case_pdf_list for item in sublist]
df_new = pd.DataFrame({'id': id_case_list, 'url':flat_url_case_pdf_list, 'details_cases':details_cases, 'downloaded':downloaded_cases})  
df_new.to_csv('/content/drive/My Drive/scriptie/metadata_cases_new.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
Error!
52
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
53
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
54
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
55
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
56
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
57
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
58
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
59
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
60
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
E

In [21]:
from pathlib import Path
new_corpus_txt = Path('/content/drive/My Drive/scriptie/corpus_archive_asylum4.txt').read_text()

In [22]:
print(new_corpus_txt[0:1000])

CASE ID: /utiac/rp-00141-2018Upper Tribunal(Immigration and Asylum Chamber)	Appeal Number: RP/00141/2018THE IMMIGRATION ACTSHeard at Field HouseDecision & Reasons PromulgatedOn the 23rd September 2022On the 4th January 2023BeforeUPPER TRIBUNAL JUDGE FRANCESDEPUTY UPPER TRIBUNAL JUDGE COTTONBetweenM M S(anonymity direction MADE)AppellantandTHE SECRETARY OF STATE FOR THE HOME DEPARTMENTRespondentRepresentation:For the Appellant:	Mr E Nicholson, instructed by Thompson & Co SolicitorsFor the Respondent:	Mr D Clarke, Senior Home Office Presenting OfficerDECISION AND REASONS1.	The appellant is a national of Somalia born in 1977. His appeal against the decision to revoke refugee status, refuse his human rights claim and deport him to Somalia was allowed by First-tier Tribunal Judge Hosie on Article 3 grounds on 7 January 2020. This decision was set aside for the reasons given in the decision of Upper Tribunal Judge Frances promulgated on 10 March 2022. The appeal was adjourned for reheari

In [23]:
corpus = re.split('CASE ID: ', new_corpus_txt)

In [24]:
len(corpus)

1726